In [1]:
#Some necessary libraries import
%tensorflow_version 1.x
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
#batch_size = number of samples that will be propagated through the network
batch_size = 128
num_classes = 10
#epoch = how many times your input will be used to update the weights while training the model
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
#build a sequential model and add convolutional layers and max pooling layers to it
model = Sequential()

#Convolutional Layer #1
model.add(Conv2D(32, kernel_size=(3, 3), #YOUR CODE HERE: applying 32 3X3 filters to the input layer
                 activation='relu', #YOUR CODE HERE: with a ReLU activation function
                 input_shape=input_shape))

#Convolutional Layer #2
model.add(Conv2D(64, (3, 3), #YOUR CODE HERE: applying 64 3X3 filters to the input layer 
                 activation='relu')) #YOUR CODE HERE: with a ReLU activation function

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## **You can track you Tensorboard logs via ngrok URL**

In [6]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-13 15:39:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.70.214.4, 52.206.78.89, 52.0.156.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.70.214.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.5MB/s    in 0.9s    

2020-04-13 15:39:01 (14.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://100b9581.ngrok.io


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

In [8]:
model.fit(x_train, y_train ,#YOUR CODE HERE
          batch_size= batch_size,
          epochs= epochs ,#YOUR CODE HERE,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples




Epoch 1/5
60000/60000 [==============================] - 15s 246us/step - loss: 0.2721 - accuracy: 0.9154 - val_loss: 0.0550 - val_accuracy: 0.9825

Epoch 2/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.0899 - accuracy: 0.9732 - val_loss: 0.0459 - val_accuracy: 0.9860
Epoch 3/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.0657 - accuracy: 0.9806 - val_loss: 0.0340 - val_accuracy: 0.9891
Epoch 4/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.0559 - accuracy: 0.9830 - val_loss: 0.0329 - val_accuracy: 0.9895
Epoch 5/5
60000/60000 [==============================] - 9s 146us/step - loss: 0.0467 - accuracy: 0.9859 - val_loss: 0.0296 - val_accuracy: 0.9900
Test loss: 0.02964161810332589
Test accuracy: 0.9900000095367432
